<font color="#CC3D3D"><p>
# OOF(Out-Of-Fold) Ensemble

##### Data Preparation ####

In [1]:
import pandas as pd
import numpy as np

loan_data = pd.read_csv('loan_train.csv')
y = loan_data['Personal Loan']
X = loan_data.drop(['ID', 'ZIP Code', 'Personal Loan'], axis=1)

In [2]:
# Kaggle에서 아래와 같이 학습데이터(X_train, y_train)와 평가데이터(X_test)를 나누어서 제공했다고 가정하자.
# 즉, y_test는 제공하지 않기 때문에 우리는 X_test에 대한 정답을 모른다.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

##### Feature Engineering ####

In [3]:
# 이 예제에서는 생략되어 있지만 반드시 Feature Engineering이 요구된다.

##### Model Building #####
*Hyperparameter Optimization*

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

model = KNeighborsClassifier()
#model = DecisionTreeClassifier()
#model = LogisticRegression()
#model = SVC()

In [5]:
# Kaggle Competition에서는 일반적으로 학습 데이터를 다시 나누지 않고 학습데이터 전체를 사용하여 model tuning을 한다.

import optuna
from sklearn.model_selection import cross_val_score

# 조절할 하이퍼 파라미터와 그 범위를 지정하는 함수 정의
def objective(trial): 
    knn_n_neighbors = trial.suggest_int('n_neighbors', 1, 10, step=1)
    knn_weights = trial.suggest_categorical('weights', ['uniform','distance'])

    classifier_obj = KNeighborsClassifier(
        n_neighbors = knn_n_neighbors, 
        weights = knn_weights,    
    )

    score = cross_val_score(classifier_obj, X_train, y_train, scoring='roc_auc', cv=5)
    accuracy = score.mean()
    return accuracy

# 최적화 실행
study = optuna.create_study(sampler=optuna.samplers.TPESampler(seed=0), direction="maximize")
study.optimize(objective, n_trials=18) 

#최적화 결과 보기
print("Best score:", study.best_value)
print("Best parameters:", study.best_params)

[I 2023-05-23 14:05:20,722] A new study created in memory with name: no-name-e1246c10-0740-47e2-8d68-ff4abc6745c1
[I 2023-05-23 14:05:20,948] Trial 0 finished with value: 0.5894245768320197 and parameters: {'n_neighbors': 6, 'weights': 'uniform'}. Best is trial 0 with value: 0.5894245768320197.
[I 2023-05-23 14:05:21,157] Trial 1 finished with value: 0.5988463694173407 and parameters: {'n_neighbors': 6, 'weights': 'distance'}. Best is trial 1 with value: 0.5988463694173407.
[I 2023-05-23 14:05:21,345] Trial 2 finished with value: 0.6003583450431436 and parameters: {'n_neighbors': 5, 'weights': 'distance'}. Best is trial 2 with value: 0.6003583450431436.
[I 2023-05-23 14:05:21,531] Trial 3 finished with value: 0.5902914964344547 and parameters: {'n_neighbors': 4, 'weights': 'uniform'}. Best is trial 2 with value: 0.6003583450431436.
[I 2023-05-23 14:05:21,746] Trial 4 finished with value: 0.5894245768320197 and parameters: {'n_neighbors': 6, 'weights': 'uniform'}. Best is trial 2 with v

Best score: 0.6018093270620579
Best parameters: {'n_neighbors': 7, 'weights': 'distance'}


*OOF Prediction*

In [6]:
# Kaggle에서는 특정모형의 과대적합을 줄이기 위해 OOF(Out-Of-Fold) Prediction을 자주 사용한다. 
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score

models = cross_validate(KNeighborsClassifier(**study.best_params), # 최적화된 hyperparameter 사용
                        X_train, y_train, cv=4, scoring='roc_auc', 
                        return_estimator=True)
oof_pred = np.array([m.predict_proba(X_test)[:,1] for m in models['estimator']]).mean(axis=0)

<img align='left' src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FpeM40%2FbtqS4NajVcL%2FwEkl1E6YsbdRMUs1yzVupK%2Fimg.png'>

In [7]:
# OOF와 일반적인 방식(sklearn 권장 방식)의 성능 비교
model = KNeighborsClassifier(**study.best_params)
model.fit(X_train, y_train)

roc_auc_score(y_test, oof_pred), roc_auc_score(y_test, model.predict_proba(X_test)[:,1])

(0.6749962619617226, 0.6510915071770335)

##### Model Ensemble ####

In [ ]:
# 이 예제에서는 생략되어 있지만, 각 모델의 OOF prediction를 이용하여 앙상블하는 것이 일반적이다.

<font color="#CC3D3D"><p>
# End